In [1]:
####### XXX

# Step 1: Sign up for a Google Earth Engine Account here:
# https://earthengine.google.com/signup/

# you will use this to authenticate your account with GEE to run the script and
# utilize GEE layers

In [1]:
# get any missing modules
! pip install earthengine-api geemap

In [15]:
# setup modules
import numpy as np
#import matplotlib.pyplot as pl
#import matplotlib as mpl
#import pandas as pd
import sys,os,os.path,tempfile

# spatial modules
try:
    import google.colab
    import geemap.foliumap as geemap # use folium instead so coloramps work on goolge collab
    # https://geemap.org/notebooks/geemap_and_folium/
except:
#    import geemap as geemap
    import geemap.foliumap as geemap
    
import ee
import geemap.colormaps as cm

import warnings
warnings.filterwarnings('ignore') #don't display warnings

In [2]:
## setup output. defined dir or system temp dir
data_dir=os.path.expanduser(os.path.join(tempfile.gettempdir(),'gee_output'))

if not os.path.exists(data_dir):
    os.makedirs(data_dir)

print(" ")
print("****** GEE and script output folder ******")
print("****** Note: In Google Collab, use the dir navigation tool on the left of the screen ******")
print("****** Use the dir navigation to go up one level, then navigate to tmp/gee_output ******")
print(data_dir)

 
****** GEE and script output folder ******
****** Note: In Google Collab, use the dir navigation tool on the left of the screen ******
****** Use the dir navigation to go up one level, then navigate to tmp/gee_output ******
/var/folders/xp/h3k9vf3n2jx181ts786_yjrn9c2gjq/T/gee_output


In [3]:
##### initialize ee API  - you need to authenticate with GEE and initialize

## Trigger the initialization flow. You only need to do this once while running notebook
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ## After inserting the API key initialize GEE
    ee.Initialize()

In [31]:
##### Get GEE datasets

### TIGER roads
roads = ee.FeatureCollection('TIGER/2016/Roads');

### Get Landsat 8 data - select scenes from a specific date range over Long Island
li_roi = ee.Geometry.Rectangle(-71.108102, 40.603476, -74.165714, 41.204794);

l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_8DAY_NDVI').filterDate('2018-07-01', '2018-09-10')\
.filterBounds(li_roi);
l8_ndvi = l8.select('NDVI');

# Maximum Value Composite
# https://developers.google.com/earth-engine/guides/ic_composite_mosaic
l8_maxval_ndvi_composite = l8_ndvi.max();

In [32]:
#### Map Landsat 8 Maximum Value Composite with RGB UAS image
ndvi_palette = cm.get_palette('viridis', n_class=30)
ndviViz = {'min': 0.0, 'max': 1.0, 'palette': ndvi_palette, 'opacity': 1}

Map = geemap.Map(center=[40.816015,-73.035151], zoom=8)
Map.add_basemap('ROADMAP') # Add Google Map
Map.addLayer(l8_maxval_ndvi_composite, ndviViz, 'Landsat 8 NDVI');
Map.add_colorbar(colors=ndviViz['palette'], vmin=ndviViz['min'], 
                     vmax=ndviViz['max'],
                     caption = "Landsat 8 NDVI (-)",
                     layer_name="Landsat 8 NDVI", 
                     position="bottomright")
#Map.addLayer(roads, {}, 'Census roads');
Map